# Welcome to the Profiling Tool for the RAPIDS Accelerator for Apache Spark
To run the tool, you need to enter a log path that represents the DBFS location for your Spark GPU event logs.  Then you can select "Run all" to execute the notebook.  After the notebook completes, you will see various output tables show up below.

## GPU Job Tuning Recommendations
This has general suggestions for tuning your applications to run optimally on GPUs.

## Per-Job Profile
The profiler output includes information about the application, data sources, executors, SQL stages, Spark properties, and key application metrics at the job and stage levels.

In [0]:
import json
import requests
import base64
import shlex
import subprocess
import pandas as pd

TOOL_JAR_URL = 'https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark-tools_2.12/22.10.0/rapids-4-spark-tools_2.12-22.10.0.jar'
TOOL_JAR_LOCAL_PATH = '/tmp/rapids-4-spark-tools.jar'

# Profiling tool output directory.
OUTPUT_DIR = '/tmp' 

response = requests.get(TOOL_JAR_URL)
open(TOOL_JAR_LOCAL_PATH, "wb").write(response.content)

In [0]:
dbutils.widgets.text("log_path", "")

In [0]:
eventlog_string=dbutils.widgets.get("log_path") 

q_command_string="java -Xmx10g -cp /tmp/rapids-4-spark-tools.jar:/databricks/jars/* com.nvidia.spark.rapids.tool.profiling.ProfileMain --csv --auto-tuner -o {} ".format(OUTPUT_DIR) + eventlog_string
args = shlex.split(q_command_string)
cmd_out = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if cmd_out.returncode != 0:
  dbutils.notebook.exit("Profiling Tool failed with stderr:" + cmd_out.stderr)

In [0]:
import os

app_df = pd.DataFrame(columns = ['appId', 'appName'])

for x in os.scandir(OUTPUT_DIR + "/rapids_4_spark_profile/"):
  tmp_df = pd.read_csv(x.path + "/application_information.csv")
  app_df = app_df.append(tmp_df[['appId', 'appName']])

## GPU Job Tuning Recommendations

In [0]:
app_list = app_df["appId"].tolist()
app_recommendations = pd.DataFrame(columns=['app', 'recommendations'])

for app in app_list:
  app_file = open(OUTPUT_DIR + "/rapids_4_spark_profile/" + app + "/profile.log")
  recommendations_start = 0
  recommendations_str = ""
  for line in app_file:
    if recommendations_start == 1:
      recommendations_str = recommendations_str + line
    if "### D. Recommended Configuration ###" in line:
      recommendations_start = 1
  app_recommendations = app_recommendations.append({'app': app, 'recommendations': recommendations_str}, ignore_index=True)
    
display(app_recommendations)

## Per-App Profile

In [0]:
for x in os.scandir(OUTPUT_DIR + "/rapids_4_spark_profile/"):
  print("APPLICATION ID = " + str(x))
  log = open(x.path + "/profile.log")
  print(log.read())